# Import

In [1]:
import numpy as np
import pandas as pd
import csv
import datetime
import math

# myLibrary

In [2]:
#製造answer.csv的第1列
def answer_header():
    header = ['user_id']
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

In [3]:
def get_week(date_str):
    d = get_date(date_str)
    w = datetime.datetime.strftime(d, '%w')
    return int(w)

In [4]:
def get_date(date_str):
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

In [5]:
def get_slot(time_str):
    the_time = datetime.datetime.strptime(time_str, '%H:%M:%S.%f')
    time_sec0 = datetime.datetime.strptime('01:00:00.00', '%H:%M:%S.%f')
    time_sec1 = datetime.datetime.strptime('09:00:00.00', '%H:%M:%S.%f')
    time_sec2 = datetime.datetime.strptime('17:00:00.00', '%H:%M:%S.%f')
    time_sec3 = datetime.datetime.strptime('21:00:00.00', '%H:%M:%S.%f')
    time_sec4 = datetime.datetime.strptime('23:59:59.99', '%H:%M:%S.%f')
    if time_sec0 <= the_time and the_time <= time_sec1:
        slot = 0
    elif time_sec1 <= the_time and the_time <= time_sec2:
        slot = 1
    elif time_sec2 <= the_time and the_time <= time_sec3:
        slot = 2
    elif time_sec3 <= the_time and the_time <= time_sec4:
        slot = 3
    else:
        slot = 4
    return slot

In [6]:
def get_days(end, start):
    d = end - start
    return d.days

In [7]:
def update_week(week, date_str, event_week, addition):
    date = get_date(date_str)
    end_date = get_date('2017-08-14')
    days = get_days(end_date, date)
    weight = 0
    for i in range(1,33):
        if (days/7) <= i:
            weight = math.pow(0.6, i)
            break
    addition = list(np.array(addition) * weight)
    week.days[event_week].add_slots(addition)
    another_addition = [0, 0, 0]
    another_addition.append(addition[4])
    week.days[event_week-1].add_slots(another_addition)
    
    yesterday_weight_addition = list(np.array(week.days[event_week-1].get_slots()) * 0.6)
    week.days[event_week].add_slots(yesterday_weight_addition)
    
    return week

In [8]:
def error_list():
    errorlist= ['error', 'api error', 'limit playzone countdown']
    return errorlist

# myClass

In [9]:
class Day():
    def __init__(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def reset(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def get_slots(self):
        return [self.slot0, self.slot1, self.slot2, self.slot3]
    def add_slots(self, addition):
        self.slot0 += addition[0]
        self.slot1 += addition[1]
        self.slot2 += addition[2]
        self.slot3 += addition[3]

In [10]:
class Week():
    def __init__(self):
        self.days = []
        for i in range(7):
            self.days.append(Day())
    def reset(self):
        for i in range(7):
            self.days[i].reset()
    def get_answer(self, id):
        tmp = []
        for i in range(1, 7):
            tmp += self.days[i].get_slots()
        tmp += self.days[0].get_slots()
        answer = [id] + list(np.array(tmp))
        return answer

# Main

In [11]:
with open('answer_v61.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(answer_header())
    week = Week()
    # 開始處理資料
    print(datetime.datetime.now())
    for data_count in range(46, 76): # revise
        # 進度顯示
        print(data_count)
        
        data = pd.read_csv('private/data-0' + str(data_count) + '.csv')
        
        # 取得目前csv檔案的首尾user_id
        head = data.head(1)['user_id'].values[0]
        tail = data.tail(1)['user_id'].values[0]
        
        # 依序處理各id資料並寫入answer.csv
        for id in range(head, tail+1):
            id_data = data[data['user_id'] == id]
            length = len(id_data)
            
            # 依序處理某id的每一筆資料
            now_date = id_data['event_time'].values[0].split()[0]
            addition = [0, 0, 0, 0, 0]
            for i in range(length):
                # 切分時間獲得星期和slot
                event_time = id_data['event_time'].values[i].split()
                date = event_time[0]
                
                # 當日期變化
                if date != now_date:
                    # 將前日的紀錄更新進week
                    event_week = get_week(now_date)
                    week = update_week(week, now_date, event_week, addition)
                    #並初始化now_date和addition
                    now_date = date
                    addition = [0, 0, 0, 0, 0]
                
                # 若無error則把紀錄活動至addition
                trigger = id_data['action_trigger'].values[i]
                if trigger not in error_list():
                    event_slot = get_slot(event_time[1])
                    addition[event_slot] = 1
            
            # 補齊最後一段日期的紀錄
            event_week = get_week(now_date)
            week = update_week(week, now_date, event_week, addition)
            
            # 寫入answer.csv
            writer.writerow(week.get_answer(id))
            
            # 初始化week
            week.reset()
            
            #break # revise
            
    print(datetime.datetime.now())
    
# test

2018-12-05 13:02:42.372656
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
2018-12-05 13:51:39.439613


# Test Area

In [12]:
id_data = data[data['user_id'] == 57159]
id_data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer


In [13]:
data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
0,93913,188172,9206492,16,2017-01-05 14:01:01.248,2,interrupt,iOS,16,17,wifi,False
1,93913,188172,9206492,16,2017-01-05 14:02:06.008,55,leave,iOS,17,17,wifi,False
2,93913,188172,9206492,16,2017-01-05 14:19:23.126,1029,interrupt,iOS,16,17,wifi,False
3,93913,188172,9206492,16,2017-01-05 15:02:57.054,2587,leave,iOS,16,17,wifi,False
4,93913,188172,9206492,16,2017-01-05 15:19:36.894,992,interrupt,iOS,17,17,wifi,False
5,93913,188172,9206492,16,2017-01-05 15:21:17.121,7,seek,iOS,17,17,wifi,False
6,93913,188172,9206492,16,2017-01-05 15:21:21.060,2,seek,iOS,17,17,wifi,False
7,93913,188172,9206492,16,2017-01-05 15:21:25.139,2,seek,iOS,17,17,wifi,False
8,93913,188172,9206492,16,2017-01-05 15:51:49.170,1817,seek,iOS,17,17,wifi,False
9,93913,188172,9206492,16,2017-01-05 15:51:53.570,3,seek,iOS,17,17,wifi,False


# Check Answer

In [14]:
answer = pd.read_csv('answer_v61.csv')
answer

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,15.962021,16.706736,1.185014e+01,21.025894,1.751672e+01,1.873312e+01,1.289448e+01,22.781712,1.642632e+01,...,1.712560e+01,27.573425,1.396454e+01,13.966759,1.071881e+01,18.741246,24.606482,2.509322e+01,1.907350e+01,32.458702
1,57160,0.718239,2.575990,9.435483e-01,0.116095,7.770945e-01,3.311499e+00,8.717854e-01,0.093602,1.983879e+00,...,4.165818e-01,0.100777,5.900056e-01,1.060060,5.040953e-01,0.100777,0.736654,1.611800e+00,1.064642e+00,0.126979
2,57161,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000,2.160000e-01,0.000000,0.000000e+00,0.360000,0.000000,0.000000e+00,0.000000e+00,0.000000
3,57162,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,6.000000e-01,0.000000e+00,0.000000,0.000000e+00,...,1.039576e+00,0.000000,6.046618e-03,0.066450,1.713328e-01,0.000000,0.005805,2.593529e-01,3.201057e-01,0.000000
4,57163,0.360000,0.360000,0.000000e+00,0.000000,5.760000e-01,5.760000e-01,0.000000e+00,0.000000,3.456000e-01,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
5,57164,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.360000,0.000000e+00,0.000000,0.000000e+00,0.216000,0.000000,2.160000e-01,0.000000e+00,0.129600
6,57165,1.155274,2.800529,9.312454e-01,0.789614,7.991887e-01,3.758963e+00,1.506869e+00,1.279001,9.902776e-02,...,7.497296e-01,0.996019,5.828711e-01,3.079396,1.024937e+00,0.867936,0.833332,3.079727e+00,1.123516e+00,1.104370
7,57166,0.262828,1.747941,2.937600e-01,0.000000,2.354568e-01,2.302581e+00,1.762560e-01,0.000000,1.879301e-01,...,3.600000e-01,0.000000,1.007770e-02,0.868196,2.160000e-01,0.000000,0.222047,1.097076e+00,4.896000e-01,0.000000
8,57167,13.503899,15.440211,4.377859e+00,0.433074,1.671577e+01,1.912516e+01,5.319602e+00,0.536716,2.128420e+01,...,1.134486e+01,1.063933,3.246782e+01,37.523427,1.100946e+01,1.042729,32.285593,3.672191e+01,1.060456e+01,1.013035
9,57168,42.481004,40.539033,0.000000e+00,57.597723,4.514895e+01,4.346012e+01,0.000000e+00,60.036489,6.353729e+01,...,0.000000e+00,131.582841,2.541328e+01,24.078220,0.000000e+00,33.826834,25.931288,2.452479e+01,0.000000e+00,35.854840
